In [1]:
import pystac_client
import pystac
import odc.stac
import geopandas
import xarray 
import rioxarray
import planetary_computer
import pathlib
import pandas
import numpy
import folium
import plotly.express
import geoviews
import geoviews.feature
import cartopy 
import dask.distributed
import branca.element, branca.colormap # Remove whitespace around small folium map
import bokeh
import hvplot.xarray

module_path = pathlib.Path.cwd() / '..' / 'scripts'
import sys
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
import utils

geoviews.extension('bokeh', 'matplotlib')

%load_ext autoreload
%autoreload 2


# Ensure geometries setup

In [2]:
tiles_south_island = utils.create_tiles_south_island(distance_offshore = 3_000, tile_length = 10_000)
tiles_chatham_island = utils.create_tiles_chatham_island(distance_offshore = 3_000, tile_length = 10_000)
tiles_north_island = utils.create_tiles_north_island(distance_offshore = 3_000, tile_length = 10_000)
test_sites = utils.create_test_sites(distance_offshore = 3_000)
land = geopandas.read_file(utils.DATA_PATH / "vectors" / f"main_islands.gpkg")

# Set up - call only once each session

In [3]:
client = dask.distributed.Client()
odc.stac.configure_rio(cloud_defaults=True, client=client)
display(client)

odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
catalogue = {"url": "https://planetarycomputer.microsoft.com/api/stac/v1",
             "collections": {"sentinel": "sentinel-2-l2a", "dem": "cop-dem-glo-30"}}
client = pystac_client.Client.open(catalogue["url"], modifier=planetary_computer.sign_inplace) 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 31.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49903,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.73 GiB
Comm: tcp://127.0.0.1:49922,Total threads: 2
Dashboard: http://127.0.0.1:49924/status,Memory: 7.93 GiB
Nanny: tcp://127.0.0.1:49906,


# Site of interest

In [4]:
date_YYMM = "2016-09-19" # "2017-02"
site_name = "rakiora"
site_index = (test_sites["name"] == site_name).idxmax()
site_bbox = test_sites.to_crs(utils.CRS_WSG).loc[site_index].geometry.bounds

output_path = utils.DATA_PATH / "rasters" / "test_sites" / site_name
output_path.mkdir(parents=True, exist_ok=True)

# Default values

In [5]:
data = utils.kelp_single_day(client, date_YYMM, roi=test_sites.loc[[site_index]])
kelp_polygons = utils.polygon_from_raster(data["kelp"])

		Ocean cloud percentage ['0.62%']


In [28]:
buffer = 3
kelp_polygons_buffered = kelp_polygons.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer).buffer(utils.RASTER_DEFAULTS["resolution"] * buffer)
ocean_buffered = kelp_polygons.overlay(test_sites.loc[[site_index]].to_crs(kelp_polygons.crs), how="difference")#.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer)

In [ ]:
# Put it all on a single interactive map
center = [(site_bbox[1] + site_bbox[3]) / 2, (site_bbox[0] + site_bbox[2]) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=13)

# RGB
data.odc.to_rgba(bands=utils.get_band_names_from_common(["red", "green", "blue"]), vmin=0, vmax=1000).odc.add_to(m, name="Satellite RBG")


# Categorical for the predicted classes and for the training data
data["kelp"].odc.add_to(m, name="Kelp")

kelp_polygons.explore(m=m, color="magenta", style_kwds={"fillOpacity": 0})
kelp_polygons_buffered.explore(m=m, color="red", style_kwds={"fillOpacity": 0})

# Layer control
folium.LayerControl().add_to(m)

m

In [29]:
buffer = 3
kelp_polygons_buffered = kelp_polygons.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer).buffer(utils.RASTER_DEFAULTS["resolution"] * buffer)
ocean_buffered = kelp_polygons.overlay(test_sites.loc[[site_index]].to_crs(kelp_polygons.crs), how="difference")#.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer)

m=kelp_polygons_buffered.explore()
ocean_buffered.explore(m=m, color="magenta")
test_sites.loc[[site_index]].explore(color="g")
m